### Importing libraries

In [17]:
#Importing image file paths
import os
import glob
import time

#For managing dataframe
import pandas as pd
import numpy as np

#PyTorch
import torchaudio
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import Dataset
from torch import nn
from torchvision import transforms


#Metrics
import seaborn as sb
import matplotlib.pyplot as plt
from torchmetrics.classification import MulticlassF1Score
from torchmetrics.classification import ConfusionMatrix

# Misc.
import warnings
import torchvision
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
from tqdm.notebook import tqdm_notebook
from collections import defaultdict

import torch.nn.functional as F
from torchvision import transforms
import glob
import time


warnings.filterwarnings("ignore")

#Selecting device
device="cuda" if torch.cuda.is_available() else "cpu"

print(f"Using {device}")

Using cuda


### Dataset

In [2]:
class Dataset(torch.utils.data.IterableDataset):
    def __init__(self, path, shuffle_pairs=True, augment=False):
        '''
        Create an iterable dataset from a directory containing sub-directories of 
        entities with their images contained inside each sub-directory.

            Parameters:
                    path (str):                 Path to directory containing the dataset.
                    shuffle_pairs (boolean):    Pass True when training, False otherwise. When set to false, the image pair generation will be deterministic
                    augment (boolean):          When True, images will be augmented using a standard set of transformations.

            where b = batch size

            Returns:
                    output (torch.Tensor): shape=[b, 1], Similarity of each pair of images
        '''
        self.path = path

        self.feed_shape = [3, 224, 224]
        self.shuffle_pairs = shuffle_pairs

        self.augment = augment

        if self.augment:
            # If images are to be augmented, add extra operations for it (first two).
            self.transform = transforms.Compose([
                transforms.RandomAffine(degrees=20, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=0.2),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                transforms.Resize(self.feed_shape[1:])
            ])
        else:
            # If no augmentation is needed then apply only the normalization and resizing operations.
            self.transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                transforms.Resize(self.feed_shape[1:])
            ])

        self.create_pairs()

    def create_pairs(self):
        '''
        Creates two lists of indices that will form the pairs, to be fed for training or evaluation.
        '''

        self.image_paths = glob.glob(os.path.join(self.path, "*/*.jpg"))
        self.image_classes = []
        self.class_indices = {}

        for image_path in self.image_paths:
            image_class = image_path.split(os.path.sep)[-2]
            self.image_classes.append(image_class)

            if image_class not in self.class_indices:
                self.class_indices[image_class] = []
            self.class_indices[image_class].append(self.image_paths.index(image_path))

        self.indices1 = np.arange(len(self.image_paths))

        if self.shuffle_pairs:
            np.random.seed(int(time.time()))
            np.random.shuffle(self.indices1)
        else:
            # If shuffling is set to off, set the random seed to 1, to make it deterministic.
            np.random.seed(1)

        select_pos_pair = np.random.rand(len(self.image_paths)) < 0.5

        self.indices2 = []

        for i, pos in zip(self.indices1, select_pos_pair):
            class1 = self.image_classes[i]
            if pos:
                class2 = class1
            else:
                class2 = np.random.choice(list(set(self.class_indices.keys()) - {class1}))
            idx2 = np.random.choice(self.class_indices[class2])
            self.indices2.append(idx2)
        self.indices2 = np.array(self.indices2)

    def __iter__(self):
        self.create_pairs()

        for idx, idx2 in zip(self.indices1, self.indices2):

            image_path1 = self.image_paths[idx]
            image_path2 = self.image_paths[idx2]

            class1 = self.image_classes[idx]
            class2 = self.image_classes[idx2]

            image1 = Image.open(image_path1).convert("RGB")
            image2 = Image.open(image_path2).convert("RGB")

            if self.transform:
                image1 = self.transform(image1).float()
                image2 = self.transform(image2).float()

            yield (image1, image2), torch.FloatTensor([class1==class2]), (class1, class2)
        
    def __len__(self):
        return len(self.image_paths)


In [23]:
image_path=[]
image_fetch("../DATASET/",image_path)

In [24]:
d1=defaultdict(int)
for i in range(len(image_path)):
    d1[image_path[i].split("/")[-2]]+=1

In [27]:
d1

defaultdict(int,
            {'NOTEBOOK': 15,
             'LUNCH BOX': 4,
             'PEN1': 7,
             'PHONE': 11,
             'HEADPHONES': 3,
             'METAL BOTTLE': 8,
             'PLASTIC BOTTLE': 9,
             'WALLET': 15,
             'PEN4': 4,
             'MOBILE CHARGER': 5,
             'PENCIL KIT 2': 16,
             'HEADPHONES 2': 3,
             'PEN3': 5,
             'KEYS 2': 3,
             'LAPTOP CHARGER': 3,
             'CALCULATOR 2': 4,
             'AIRPODS': 7,
             'KEYS': 8,
             'TEXTBOOK': 10,
             'WALLET 2': 5,
             'PEN2': 7,
             'PENCIL': 6,
             'SCALE': 9,
             'BAG': 4,
             'CALCULATOR': 5,
             'PENCIL KIT': 3,
             'ID CARD FULL': 4,
             'EARPHONES': 5})

In [ ]:
class SaimeseNetworkDataset(Dataset):
    def __init__(self, folder_path):

        self.image_path=[]
        image_fetch(folder_path,self.image_path)

        self.class_count=defaultdict(int)
    
        for i in range(len(self.image_path)):
            self.class_count[self.image_path[i].split("/")[-2]]+=1

        
    

        

    def image_fetch(src,image_paths,file=None):
        l=os.listdir(src)
        if(len(l)!=0):
            for i in range(len(l)):
                if(file!=None):
                    if(file in l[i]):
                        image_paths.append(str(src+l[i]))
                elif("." not in l[i]):
                    try:
                        image_fetch(str(src+"/"+l[i]+"/"),image_paths)
                    except:
                        continue
                else:
                    if(".jpeg" in l[i] or ".png" in l[i] or ".jpg" in l[i]):
                        image_paths.append(str(src+l[i]))
                            
    
    def __len__(self):

    def __getitem__(self,index):
        

In [ ]:
l1=[]

In [40]:
for path in image_path:
    #class_obj=path.split(path.split("/")[-1])[0]
    class_obj=path.split("/")[-2]
    
    count=d1[class_obj]

    i=count
    while(count>0):
        
        i-=1
    

NOTEBOOK
15
NOTEBOOK
15
NOTEBOOK
15
NOTEBOOK
15
NOTEBOOK
15
NOTEBOOK
15
NOTEBOOK
15
NOTEBOOK
15
NOTEBOOK
15
NOTEBOOK
15
NOTEBOOK
15
NOTEBOOK
15
NOTEBOOK
15
NOTEBOOK
15
NOTEBOOK
15
LUNCH BOX
4
LUNCH BOX
4
LUNCH BOX
4
LUNCH BOX
4
PEN1
7
PEN1
7
PEN1
7
PEN1
7
PEN1
7
PEN1
7
PEN1
7
PHONE
11
PHONE
11
PHONE
11
PHONE
11
PHONE
11
PHONE
11
PHONE
11
PHONE
11
PHONE
11
PHONE
11
PHONE
11
HEADPHONES
3
HEADPHONES
3
HEADPHONES
3
METAL BOTTLE
8
METAL BOTTLE
8
METAL BOTTLE
8
METAL BOTTLE
8
METAL BOTTLE
8
METAL BOTTLE
8
METAL BOTTLE
8
METAL BOTTLE
8
PLASTIC BOTTLE
9
PLASTIC BOTTLE
9
PLASTIC BOTTLE
9
PLASTIC BOTTLE
9
PLASTIC BOTTLE
9
PLASTIC BOTTLE
9
PLASTIC BOTTLE
9
PLASTIC BOTTLE
9
PLASTIC BOTTLE
9
WALLET
15
WALLET
15
WALLET
15
WALLET
15
WALLET
15
WALLET
15
WALLET
15
WALLET
15
WALLET
15
WALLET
15
WALLET
15
WALLET
15
WALLET
15
WALLET
15
WALLET
15
PEN4
4
PEN4
4
PEN4
4
PEN4
4
MOBILE CHARGER
5
MOBILE CHARGER
5
MOBILE CHARGER
5
MOBILE CHARGER
5
MOBILE CHARGER
5
PENCIL KIT 2
16
PENCIL KIT 2
16
PENCIL KIT 2
16
P

In [29]:
d1

defaultdict(int,
            {'NOTEBOOK': 15,
             'LUNCH BOX': 4,
             'PEN1': 7,
             'PHONE': 11,
             'HEADPHONES': 3,
             'METAL BOTTLE': 8,
             'PLASTIC BOTTLE': 9,
             'WALLET': 15,
             'PEN4': 4,
             'MOBILE CHARGER': 5,
             'PENCIL KIT 2': 16,
             'HEADPHONES 2': 3,
             'PEN3': 5,
             'KEYS 2': 3,
             'LAPTOP CHARGER': 3,
             'CALCULATOR 2': 4,
             'AIRPODS': 7,
             'KEYS': 8,
             'TEXTBOOK': 10,
             'WALLET 2': 5,
             'PEN2': 7,
             'PENCIL': 6,
             'SCALE': 9,
             'BAG': 4,
             'CALCULATOR': 5,
             'PENCIL KIT': 3,
             'ID CARD FULL': 4,
             'EARPHONES': 5})

In [66]:
import torch.nn.functional as F
from torchvision import transforms
import glob
import time

class Dataset(torch.utils.data.IterableDataset):
    def __init__(self, path, shuffle_pairs=True, augment=False):
        '''
        Create an iterable dataset from a directory containing sub-directories of 
        entities with their images contained inside each sub-directory.

            Parameters:
                    path (str):                 Path to directory containing the dataset.
                    shuffle_pairs (boolean):    Pass True when training, False otherwise. When set to false, the image pair generation will be deterministic
                    augment (boolean):          When True, images will be augmented using a standard set of transformations.

            where b = batch size

            Returns:
                    output (torch.Tensor): shape=[b, 1], Similarity of each pair of images
        '''
        self.path = path

        self.feed_shape = [3, 224, 224]
        self.shuffle_pairs = shuffle_pairs

        self.augment = augment

        if self.augment:
            # If images are to be augmented, add extra operations for it (first two).
            self.transform = transforms.Compose([
                transforms.RandomAffine(degrees=20, translate=(0.2, 0.2), scale=(0.8, 1.2), shear=0.2),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                transforms.Resize(self.feed_shape[1:])
            ])
        else:
            # If no augmentation is needed then apply only the normalization and resizing operations.
            self.transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                transforms.Resize(self.feed_shape[1:])
            ])

        self.create_pairs()

    def create_pairs(self):
        '''
        Creates two lists of indices that will form the pairs, to be fed for training or evaluation.
        '''

        self.image_paths = glob.glob(os.path.join(self.path, "*/*.jpg"))
        self.image_classes = []
        self.class_indices = {}

        for image_path in self.image_paths:
            image_class = image_path.split(os.path.sep)[-2]
            self.image_classes.append(image_class)

            if image_class not in self.class_indices:
                self.class_indices[image_class] = []
            self.class_indices[image_class].append(self.image_paths.index(image_path))

        self.indices1 = np.arange(len(self.image_paths))

        if self.shuffle_pairs:
            np.random.seed(int(time.time()))
            np.random.shuffle(self.indices1)
        else:
            # If shuffling is set to off, set the random seed to 1, to make it deterministic.
            np.random.seed(1)

        select_pos_pair = np.random.rand(len(self.image_paths)) < 0.5

        self.indices2 = []

        for i, pos in zip(self.indices1, select_pos_pair):
            class1 = self.image_classes[i]
            if pos:
                class2 = class1
            else:
                class2 = np.random.choice(list(set(self.class_indices.keys()) - {class1}))
            idx2 = np.random.choice(self.class_indices[class2])
            self.indices2.append(idx2)
        self.indices2 = np.array(self.indices2)

    def __iter__(self):
        self.create_pairs()

        for idx, idx2 in zip(self.indices1, self.indices2):

            image_path1 = self.image_paths[idx]
            image_path2 = self.image_paths[idx2]

            class1 = self.image_classes[idx]
            class2 = self.image_classes[idx2]

            image1 = Image.open(image_path1).convert("RGB")
            image2 = Image.open(image_path2).convert("RGB")

            if self.transform:
                image1 = self.transform(image1).float()
                image2 = self.transform(image2).float()

            yield (image1, image2), torch.FloatTensor([class1==class2]), (class1, class2)
        
    def __len__(self):
        return len(self.image_paths)

In [79]:
train_dataset=Dataset('../DATASET/')

In [80]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, drop_last=True)


In [81]:
for (img1, img2), y, (class1, class2) in train_dataloader:
    print(class1,class2,y)

('ID CARD FULL', 'AIRPODS', 'MOBILE CHARGER', 'NOTEBOOK', 'MOBILE CHARGER', 'PLASTIC BOTTLE', 'WALLET 2', 'NOTEBOOK') ('ID CARD FULL', 'PENCIL KIT', 'NOTEBOOK', 'PEN4', 'PEN3', 'PLASTIC BOTTLE', 'PEN2', 'KEYS') tensor([[1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.]])


KeyboardInterrupt: 